# Version VIII - Ensemble ColorOperations (10 models, Color operations Data Augmentation)

### Imports and initial setup

In [1]:
import import_ipynb

from InitialSetup import dataset, dataset_length, BATCH_SIZE, tf, prepare_callbacks, model_VI, show_history, test_dataset, test_dataset_length, AUTOTUNE, train_models, load_weights, load_weights_read, evaluate_models, get_labels_logits_and_preds, get_class_preds, get_class_from_sum_of_logits, np, BatchNormalization, LeakyReLU, process_hue, process_contrast, process_brightness, process_saturation

NUM_MODELS = 10

importing Jupyter notebook from InitialSetup.ipynb
Num GPUs Available:  1
Total images in validatation dataset:  12630
(32, 32, 3) (43,)
Total images in dataset:  39209


### Prepare dataset

In [8]:
BATCH_SIZE = 64

dataSolo = dataset
dataSolo = dataSolo.concatenate(dataset.map(process_hue))
dataSolo = dataSolo.concatenate(dataset.map(process_contrast))
dataSolo = dataSolo.concatenate(dataset.map(process_brightness))
dataSolo = dataSolo.concatenate(dataset.map(process_saturation))

dataSolo_size = tf.data.experimental.cardinality(dataSolo).numpy()

dataSolo = dataSolo.cache()
dataSolo = dataSolo.shuffle(buffer_size = dataSolo_size)
dataSolo = dataSolo.batch(batch_size = BATCH_SIZE)
dataSolo = dataSolo.prefetch(buffer_size = AUTOTUNE)

train_size = int(0.8* dataSolo_size)
val_size = int(0.2* dataSolo_size)

train_dataset = dataSolo.take(train_size)
val_dataset = dataSolo.skip(train_size)

file_path_prefix = './Networks/ensembles/V1/V1'    

### Setup models and save folder

In [3]:
models_V1, histories_V1 = train_models(dataSolo, train_size/BATCH_SIZE, val_dataset, val_size/BATCH_SIZE, file_path_prefix, NUM_MODELS)

Epoch 1/20
2450/2450 [==============================] - 108s 29ms/step - loss: 2.0582 - accuracy: 0.5244 - val_loss: 0.0979 - val_accuracy: 0.9763

Epoch 00001: val_accuracy improved from -inf to 0.97635, saving model to ./Networks/ensembles/V1/V1_00\cp.ckpt
Epoch 2/20
2450/2450 [==============================] - 71s 29ms/step - loss: 0.2289 - accuracy: 0.9337 - val_loss: 0.0450 - val_accuracy: 0.9888

Epoch 00002: val_accuracy improved from 0.97635 to 0.98881, saving model to ./Networks/ensembles/V1/V1_00\cp.ckpt
Epoch 3/20
2450/2450 [==============================] - 70s 29ms/step - loss: 0.1149 - accuracy: 0.9665 - val_loss: 0.0332 - val_accuracy: 0.9913

Epoch 00003: val_accuracy improved from 0.98881 to 0.99126, saving model to ./Networks/ensembles/V1/V1_00\cp.ckpt
Epoch 4/20
2450/2450 [==============================] - 70s 28ms/step - loss: 0.0769 - accuracy: 0.9768 - val_loss: 0.0303 - val_accuracy: 0.9908

Epoch 00004: val_accuracy did not improve from 0.99126
Epoch 5/20
2450/2

### Load Weights and evaluate models

In [4]:
models_V0 = load_weights_read(file_path_prefix, NUM_MODELS)
evaluate_models(models_V0, NUM_MODELS)

198/198 - 3s - loss: 0.0884 - accuracy: 0.9847
198/198 - 3s - loss: 0.1108 - accuracy: 0.9820
198/198 - 3s - loss: 0.0828 - accuracy: 0.9865
198/198 - 2s - loss: 0.0920 - accuracy: 0.9846
198/198 - 2s - loss: 0.0846 - accuracy: 0.9867
198/198 - 2s - loss: 0.0994 - accuracy: 0.9838
198/198 - 2s - loss: 0.0958 - accuracy: 0.9838
198/198 - 3s - loss: 0.0775 - accuracy: 0.9856
198/198 - 3s - loss: 0.0831 - accuracy: 0.9845
198/198 - 3s - loss: 0.0873 - accuracy: 0.9851
average accuracy: 98.473


In [5]:
labels_V0, logits_V0, preds_V0 = get_labels_logits_and_preds(models_V0, NUM_MODELS)
class_preds_V0 = get_class_preds(preds_V0, NUM_MODELS)
class_logits_V0 = get_class_from_sum_of_logits(logits_V0, NUM_MODELS)

### Build list of inputs and labels for test class

In [6]:
test_logits_preds = []

for i in range(test_dataset_length):
    
    aux = []   
    for m in range(NUM_MODELS):        
        aux.extend(logits_V0[m][i])
        
    test_logits_preds.append(aux)

### Compute logit predictions and training labels

In [9]:
dataSolo_len = len(dataSolo.take(-1))
j=0

logits_train = [[] for _ in range(NUM_MODELS)]
labels_aux = []
for images, labs in dataSolo.take(dataSolo_len):

    #Printing progress
    print('\r' + f'{j}/{dataSolo_len}',end="")
    j+=1
    
    labels_aux.extend(labs.numpy())
    for i in range(NUM_MODELS):
        
        logits_train[i].extend(models_V0[i][1].predict(images))
        
labels_train = [np.argmax(i) for i in labels_aux]  

3063/3064

### Build list of train inputs

In [12]:
train_logits_preds = []

for i in range(dataSolo_size):
    
    aux = []
    
    for m in range(NUM_MODELS):
        
        aux.extend(logits_train[m][i])
        
    train_logits_preds.append(aux)

In [13]:
print(len(labels_train), len(train_logits_preds))

196045 196045


In [14]:

stack_model  = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(len(train_logits_preds[0]),)),

  tf.keras.layers.Dense(256),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(64),    
  BatchNormalization(),LeakyReLU(alpha=0.01),
  tf.keras.layers.Dropout(0.4),

  tf.keras.layers.Dense(43, activation='softmax')
])

In [15]:
stack_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy',metrics=['accuracy'])

file_pathV7 = './Networks/ensembles/V1/stacked_V1/StackedColorOpsDataAug.ckpt'

callbacksV7 = prepare_callbacks(file_pathV7)

In [16]:
print(len(labels_train), len(train_logits_preds), len(logits_train[0]))

196045 196045 196045


In [17]:
stack_model.fit(np.asarray(train_logits_preds),np.asarray(labels_train),epochs=20, batch_size=BATCH_SIZE,
                        validation_data = (np.asarray(test_logits_preds), np.asarray(labels_V0)), callbacks = callbacksV7)

Epoch 1/20
3064/3064 [==============================] - 22s 7ms/step - loss: 2.0382 - accuracy: 0.5466 - val_loss: 0.2106 - val_accuracy: 0.9745

Epoch 00001: val_accuracy improved from -inf to 0.97451, saving model to ./Networks/ensembles/V1/stacked_V1\StackedColorOpsDataAug.ckpt
Epoch 2/20
3064/3064 [==============================] - 21s 7ms/step - loss: 0.4022 - accuracy: 0.9233 - val_loss: 0.0819 - val_accuracy: 0.9829

Epoch 00002: val_accuracy improved from 0.97451 to 0.98290, saving model to ./Networks/ensembles/V1/stacked_V1\StackedColorOpsDataAug.ckpt
Epoch 3/20
3064/3064 [==============================] - 21s 7ms/step - loss: 0.1897 - accuracy: 0.9620 - val_loss: 0.0662 - val_accuracy: 0.9844

Epoch 00003: val_accuracy improved from 0.98290 to 0.98440, saving model to ./Networks/ensembles/V1/stacked_V1\StackedColorOpsDataAug.ckpt
Epoch 4/20
3064/3064 [==============================] - 20s 7ms/step - loss: 0.1322 - accuracy: 0.9710 - val_loss: 0.0609 - val_accuracy: 0.9851

Ep

In [18]:
stack_model.load_weights(file_pathV7)

In [19]:
pred = stack_model.predict(np.asarray(test_logits_preds))

correct = 0

for i in range(test_dataset_length):
    if np.argmax(pred[i]) == labels_V0[i] :
        correct += 1
        
print(correct, correct/test_dataset_length)

12472 0.9874901029295329
